# ANLY561 Homework 12
# Hongyang Zheng

<br>
### Question1
###### a)

Prepare and define some functions

In [ ]:
import numpy as np
import numpy.random as rd
from sklearn.datasets import load_breast_cancer


def chain_rule(Dg, Df, var_shape):
    dim = len(var_shape)
    Dg_axes = list(range(Dg.ndim - dim, Dg.ndim))
    Df_axes = list(range(dim))
    return np.tensordot(Dg, Df, axes=(Dg_axes, Df_axes))


def DX_affine(X, W, b):
    D = np.zeros((X.shape[0], W.shape[1], X.shape[0], X.shape[1]))
    for k in range(X.shape[0]):
        D[k,:,k,:] = W.T
    return D, X.shape


def DW_affine(X, W, b):
    D = np.zeros((X.shape[0], W.shape[1], W.shape[0], W.shape[1]))
    for k in range(W.shape[1]):
        D[:,k,:,k] = X
    return D, W.shape


def Db_affine(X, W, b):
    D = np.zeros((X.shape[0], W.shape[1], b.shape[1]))
    for k in range(b.shape[1]):
        D[:,k,k] = 1
    return D, b.shape

In [1]:
def psi3(X, var):
    return matrix_softmax(
        logit(logit(X @ var[0] + var[1])@ var[2] + var[3])@ var[4] + var[5])

    
def logit(z):
    return 1 / (1 + np.exp(-z))


def Dlogit(Z):
    D = np.zeros((Z.shape[0], Z.shape[1], Z.shape[0], Z.shape[1]))
    A = logit(Z) * logit(-Z)
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            D[i,j,i,j] = A[i,j]
    return D, Z.shape


def softmax(z):
    v = np.exp(z)
    return v / np.sum(v)


def matrix_softmax(Z):
    return np.apply_along_axis(softmax, 1, Z)


def Dmatrix_softmax(Z):
    D = np.zeros((Z.shape[0], Z.shape[1], Z.shape[0], Z.shape[1]))
    for k in range(Z.shape[0]):
        v = np.exp(Z[k,:])
        v = v / np.sum(v)
        D[k,:,k,:] = np.diag(v) - np.outer(v, v)
    return D, Z.shape


def cross_entropy(P, Q):
    return -np.sum(P * np.log(Q)) / P.shape[0]


def DQcross_entropy(P, Q):
    return - P * (1 / Q) / P.shape[0], Q.shape

In [2]:
def nn_loss_closure(X, Y):
    f = lambda var: cross_entropy(Y, psi3(X, var))
    return f


def nn_loss_gradient_closure(X, Y):
    def df(var):
        # 1st hidden layer
        Z1 = (X @ var[0]) + var[1]
        X2 = logit(Z1)
        
        # 2nd hidden layer
        Z2 = (X2 @ var[2]) + var[3]
        X3 = logit(Z2)
        
        # output layer
        Z3 = (X3 @ var[4]) + var[5]
        Q = matrix_softmax(Z3)
        
        D_Q, Qshape = DQcross_entropy(Y, Q)
        D_Z3, Z3shape = Dmatrix_softmax(Z3)
        back_prop3 = chain_rule(D_Q, D_Z3, Qshape)
        
        D_X3, X3shape = DX_affine(X3, var[4], var[5])
        D_W3, W3shape = DW_affine(X3, var[4], var[5])
        D_b3, b3shape = Db_affine(X3, var[4], var[5])
        
        D_Z2, Z2shape = Dlogit(Z2)
        back_prop2 = chain_rule(
            chain_rule(back_prop3, D_X3, X3shape),
            D_Z2, 
            Z2shape
        )
        
        D_X2, X2shape = DX_affine(X2, var[2], var[3])
        D_W2, W2shape = DW_affine(X2, var[2], var[3])
        D_b2, b2shape = Db_affine(X2, var[2], var[3])
        
        D_Z1, Z1shape = Dlogit(Z1)
        back_prop1 = chain_rule(
            chain_rule(back_prop2, D_X2, X2shape), 
            D_Z1, 
            Z1shape
        )
        
        D_W1, W1shape = DW_affine(X, var[0], var[1])
        D_b1, b1shape = Db_affine(X, var[0], var[1])
        
        W1grad = chain_rule(back_prop1, D_W1, W1shape)
        b1grad = chain_rule(back_prop1, D_b1, b1shape)
        W2grad = chain_rule(back_prop2, D_W2, W2shape)
        b2grad = chain_rule(back_prop2, D_b2, b2shape)
        W3grad = chain_rule(back_prop3, D_W3, W3shape)
        b3grad = chain_rule(back_prop3, D_b3, b3shape)
        
        return [W1grad, b1grad, W2grad, b2grad, W3grad, b3grad]
    return df

In [3]:
def update_blocks(x, y, t):
    num_blocks = len(x)
    z = [None] * num_blocks
    for i in range(num_blocks):
        z[i] = x[i] + t * y[i]
    return z
      
    
def block_backtracking(x0, f, dx, df0, alpha=0.1, beta=0.5):
    num_blocks = len(x0)
    
    delta = 0
    for i in range(num_blocks):
        delta = delta + np.sum(dx[i] * df0[i])
    delta = alpha * delta
    
    f0 = f(x0)
    
    t = 1
    x = update_blocks(x0, dx, t)
    fx = f(x)
    while (not np.isfinite(fx)) or f0 + t * delta < fx:
        t = beta * t
        x = update_blocks(x0, dx, t)
        fx = f(x)
        
    return x, fx


def negate_blocks(x):
    num_blocks = len(x)
    z = [None] * num_blocks
    for i in range(num_blocks):
        z[i] = -x[i]
    return z


def block_norm(x):
    num_blocks=len(x)
    z = 0
    for i in range(num_blocks):
        z = z + np.sum(x[i] ** 2)
    return np.sqrt(z)


def random_matrix(shape, sigma=0.1):
    return np.reshape(sigma * rd.randn(shape[0] * shape[1]), shape)

In [4]:
# Loads the dataset 
data = load_breast_cancer()

dim_data = 30
num_labels = 2
num_examples = 569

num_train = 400

X = data['data'] 
targets = data.target 
labels = np.zeros((num_examples, num_labels))
for i in range(num_examples):
    labels[i, targets[i]] = 1 

In [5]:
# set seed
rd.seed(1234)

# Prepare hyperparameters of the network
hidden_nodes = 20

# Initialize variables
W1_init = random_matrix((dim_data, hidden_nodes))
b1_init = np.zeros((1, hidden_nodes))

W2_init = random_matrix((hidden_nodes, hidden_nodes))
b2_init = np.zeros((1, hidden_nodes))

W3_init = random_matrix((hidden_nodes, num_labels))
b3_init = np.zeros((1, num_labels))

100 steps of gradient descent with block backtracking

In [6]:
x = [W1_init, b1_init, W2_init, b2_init, W3_init, b3_init]
f = nn_loss_closure(X[:num_train], labels[:num_train])
df = nn_loss_gradient_closure(X[:num_train], labels[:num_train])
dx = lambda v: negate_blocks(df(v))
    
for i in range(100):
    ngrad = dx(x)
    x, fval = block_backtracking(x, f, ngrad, df(x), alpha=0.1)
    
    train_data = psi3(X[:num_train], x)
    train_labels = np.argmax(train_data, axis=1)
    per_correct = 100 * (1 - np.count_nonzero(
        train_labels - targets[:num_train]) / num_train)

    if ((i % 10) == 0):
        print(
            "Step: {:2d}, ".format(i),
            "Mean loss: {:.6f}, ".format(fval),
            "Gradient norm: {:.6f}, ".format(block_norm(ngrad)),
            "Accuracy: {:.1f}".format(per_correct)
        )
    
test_data = psi3(X[num_train:], x)

test_labels = np.argmax(test_data, axis=1)
per_correct = 100 * (1 - np.count_nonzero(
    test_labels - targets[num_train:]) / (num_examples - num_train))

print('Final test accuracy: %.1f percent' % per_correct)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp


Step:  0,  Mean loss: 0.684698,  Gradient norm: 0.573653,  Accuracy: 56.8
Step: 10,  Mean loss: 0.683966,  Gradient norm: 0.001043,  Accuracy: 56.8
Step: 20,  Mean loss: 0.683963,  Gradient norm: 0.000943,  Accuracy: 56.8
Step: 30,  Mean loss: 0.683961,  Gradient norm: 0.001235,  Accuracy: 56.8
Step: 40,  Mean loss: 0.683958,  Gradient norm: 0.000846,  Accuracy: 56.8
Step: 50,  Mean loss: 0.683956,  Gradient norm: 0.001086,  Accuracy: 56.8
Step: 60,  Mean loss: 0.683953,  Gradient norm: 0.001297,  Accuracy: 56.8
Step: 70,  Mean loss: 0.683951,  Gradient norm: 0.001118,  Accuracy: 56.8
Step: 80,  Mean loss: 0.683948,  Gradient norm: 0.001184,  Accuracy: 56.8
Step: 90,  Mean loss: 0.683946,  Gradient norm: 0.001647,  Accuracy: 56.8
Final test accuracy: 76.9 percent


###### b)
1. We can reduce the size of the training set: for example, including fewer observations/rows or fewer attributes/columns.
2. We can simplify network architecture: for example, reducing the number of nodes in the network. 
3. We can use stochastic gradient descent, because it uses less memory and time than gradient descent.

### Question 2

###### a)
Assume $\mathcal{A}$ is the forth order tensor and $\mathbf{n}=\left(2,2,3,3\right)$

Calculate the convolution of X, then we have:

$$
\mathbf{X}\ast
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
=
\begin{pmatrix}
x_{1,1} & x_{1,2} & x_{1,3} \\
x_{2,1} & x_{2,2} & x_{2,3} \\
x_{3,1} & x_{3,2} & x_{3,3}
\end{pmatrix}
\ast
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
=
\begin{pmatrix}
x_{1,1}+x_{2,2} & x_{1,2}+x_{2,3} \\
x_{2,1}+x_{3,2} & x_{2,2}+x_{3,3}
\end{pmatrix},
$$

Therefore, it is a $2\times 2$ matrix.

Denote this matrix as $\mathcal{F}$ and $\mathbf{m}=\left(3,3\right)$, then we have:

$$
c_{\mathbf{i},\mathbf{j}}:\mathscr{F}_{\mathbf{n}}\times\mathscr{F}_{\mathbf{m}}\rightarrow\mathscr{F}_{\mathbf{n}_{\setminus\mathbf{i}}\oplus\mathbf{m}_{\setminus\mathbf{j}}}=\mathscr{F}_{\left(2,2\right)}.
$$


Therefore:

$$
f_{i_{1},i_{2}}=\sum_{k_{1}=1}^{3}\sum_{k_{2}=1}^{3}a_{i_{1},i_{2},k_{1},k_{2}}x_{k_{1},k_{2}}.
$$

Then we have:
$$
\begin{align*}
f_{1,1} & = x_{1,1}+x_{2,2}\implies a_{1,1,k_{1},k_{2}}=
\begin{cases}
1, & \left(k_{1},k_{2}\right)\in\left\{\left(1,1\right),\left(2,2\right)\right\} \\
0, & \text{otherwise}
\end{cases} \\
f_{1,2} & = x_{1,2}+x_{2,3}\implies a_{1,2,k_{1},k_{2}}=
\begin{cases}
1, & \left(k_{1},k_{2}\right)\in\left\{\left(1,2\right),\left(2,3\right)\right\} \\
0, & \text{otherwise}
\end{cases} \\
f_{2,1} & = x_{2,1}+x_{3,2}\implies a_{2,1,k_{1},k_{2}}=
\begin{cases}
1, & \left(k_{1},k_{2}\right)\in\left\{\left(2,1\right),\left(3,2\right)\right\} \\
0, & \text{otherwise}
\end{cases} \\
f_{2,2} & = x_{2,2}+x_{3,3}\implies a_{2,2,k_{1},k_{2}}=
\begin{cases}
1, & \left(k_{1},k_{2}\right)\in\left\{\left(2,2\right),\left(3,3\right)\right\} \\
0, & \text{otherwise}
\end{cases}.
\end{align*}
$$

Therefore, the fourth order tensor is

$$
\mathcal{A}=
\left(
\left(
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0
\end{pmatrix},
\begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0
\end{pmatrix}
\right),
\left(
\begin{pmatrix}
0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 1 & 0
\end{pmatrix},
\begin{pmatrix}
0 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}
\right)
\right).
$$

###### b)
Assume $\mathcal{A}$ is the fifth order tensor and $\mathbf{n}=\left(2,2,2,3,3\right)$

Calculate the convolution of X, then we have:
$$
\begin{align*}
& \left(
\begin{pmatrix}
x_{1,1,1} & x_{1,1,2} & x_{1,1,3} \\
x_{1,2,1} & x_{1,2,2} & x_{1,2,3} \\
x_{1,3,1} & x_{1,3,2} & x_{1,3,3}
\end{pmatrix},
\begin{pmatrix}
x_{2,1,1} & x_{2,1,2} & x_{2,1,3} \\
x_{2,2,1} & x_{2,2,2} & x_{2,2,3} \\
x_{2,3,1} & x_{2,3,2} & x_{2,3,3}
\end{pmatrix}
\right)
\ast
\left(
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix},
\begin{pmatrix}
1 & 1 \\
1 & 1
\end{pmatrix}
\right) \\
& = 
\begin{pmatrix}
x_{1,1,1} & x_{1,1,2} & x_{1,1,3} \\
x_{1,2,1} & x_{1,2,2} & x_{1,2,3} \\
x_{1,3,1} & x_{1,3,2} & x_{1,3,3}
\end{pmatrix}
\ast
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
+
\begin{pmatrix}
x_{2,1,1} & x_{2,1,2} & x_{2,1,3} \\
x_{2,2,1} & x_{2,2,2} & x_{2,2,3} \\
x_{2,3,1} & x_{2,3,2} & x_{2,3,3}
\end{pmatrix}
\ast
\begin{pmatrix}
1 & 1 \\
1 & 1
\end{pmatrix}
\end{align*}
$$

From part a), we know that the first part is:
$$
\begin{pmatrix}
x_{1,1,1} + x_{1,2,2} & x_{1,1,2} + x_{1,2,3} \\
x_{1,2,1} + x_{1,3,2} & x_{1,2,2} + x_{1,3,3}
\end{pmatrix}
$$

Therefore, the result is:
$$
\begin{pmatrix}
x_{1,1,1} + x_{1,2,2} + x_{2,1,1} + x_{2,1,2} + x_{2,2,1} + x_{2,2,2} &
x_{1,1,2} + x_{1,2,3} + x_{2,1,2} + x_{2,1,3} + x_{2,2,2} + x_{2,2,3} \\
x_{1,2,1} + x_{1,3,2} + x_{2,2,1} + x_{2,2,2} + x_{2,3,1} + x_{2,3,2} & 
x_{1,2,2} + x_{1,3,3} + x_{2,2,2} + x_{2,2,3} + x_{2,3,2} + x_{2,3,3}
\end{pmatrix}.
$$

Therefore, it is a $2\times 2$ matrix.

Denote this matrix as $\mathcal{F}$ and $\mathbf{m}=\left(2,3,3\right)$, then we have:
$$
c_{\mathbf{i},\mathbf{j}}:\mathscr{F}_{\mathbf{n}}\times\mathscr{F}_{\mathbf{m}}\rightarrow\mathscr{F}_{\mathbf{n}_{\setminus\mathbf{i}}\oplus\mathbf{m}_{\setminus\mathbf{j}}}=\mathscr{F}_{\left(2,2\right)}.
$$

And $\mathcal{F}=$
$$
f_{i_{1},i_{2}}=\sum_{k_{1}=1}^{2}\sum_{k_{2}=1}^{3}\sum_{k_{3}=1}^{3}a_{i_{1},i_{2},k_{1},k_{2},k_{3}}x_{k_{1},k_{2},k_{3}}.
$$

We know $$
f_{1,1} = \sum_{k_{1}=1}^{2}\sum_{k_{2}=1}^{3}\sum_{k_{3}=1}^{3}a_{1,1,k_{1},k_{2},k_{3}}x_{k_{1},k_{2},k_{3}}= x_{1,1,1} + x_{1,2,2} + x_{2,1,1} + x_{2,1,2} + x_{2,2,1} + x_{2,2,2},
$$

Therefore:
$$
\begin{align*}
a_{1,1,k_{1},k_{2},k_{3}} & =
\begin{cases}
1, & \left(k_{1},k_{2},k_{3}\right)\in\left\{
\left(1,1,1\right),
\left(1,2,2\right),
\left(2,1,1\right),
\left(2,1,2\right),
\left(2,2,1\right),
\left(2,2,2\right)
\right\} \\
0 , & \text{otherwise}
\end{cases} \\
a_{1,2,k_{1},k_{2},k_{3}} & =
\begin{cases}
1, & \left(k_{1},k_{2},k_{3}\right)\in\left\{
\left(1,1,2\right),
\left(1,2,3\right),
\left(2,1,2\right),
\left(2,1,3\right),
\left(2,2,2\right),
\left(2,2,3\right)
\right\} \\
0 , & \text{otherwise}
\end{cases} \\
a_{2,1,k_{1},k_{2},k_{3}} & =
\begin{cases}
1, & \left(k_{1},k_{2},k_{3}\right)\in\left\{
\left(1,2,1\right),
\left(1,3,2\right),
\left(2,2,1\right),
\left(2,2,2\right),
\left(2,3,1\right),
\left(2,3,2\right)
\right\} \\
0 , & \text{otherwise}
\end{cases} \\
a_{2,2,k_{1},k_{2},k_{3}} & =
\begin{cases}
1, & \left(k_{1},k_{2},k_{3}\right)\in\left\{
\left(1,2,2\right),
\left(1,3,3\right),
\left(2,2,2\right),
\left(2,2,3\right),
\left(2,3,2\right),
\left(2,3,3\right)
\right\} \\
0 , & \text{otherwise}
\end{cases}.
\end{align*}
$$



Thus, the fifth order tensor is

$$
\mathcal{A} =
\left(
  \left(
    \left(
      \begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0
\end{pmatrix}, \begin{pmatrix}
1 & 1 & 0 \\
1 & 1 & 0 \\
0 & 0 & 0
\end{pmatrix}
    \right),
    \left(
      \begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
0 & 0 & 0
\end{pmatrix}, \begin{pmatrix}
0 & 1 & 1 \\
0 & 1 & 1 \\
0 & 0 & 0
\end{pmatrix}
    \right)
  \right),  
  \left(
    \left(
      \begin{pmatrix}
0 & 0 & 0 \\
1 & 0 & 0 \\
0 & 1 & 0
\end{pmatrix}, \begin{pmatrix}
0 & 0 & 0 \\
1 & 1 & 0 \\
1 & 1 & 0
\end{pmatrix} 
    \right),
    \left(
      \begin{pmatrix}
0 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{pmatrix}, \begin{pmatrix}
0 & 0 & 0 \\
0 & 1 & 1 \\
0 & 1 & 1
\end{pmatrix}
    \right)
  \right)  
\right)
$$

### Question3

see the pdf document.